In [ ]:
import numpy as np
import cv2
import pandas as pd
import os
import pickle

# Face Detector

In [ ]:
# Loading all the face detection and descriptor models
face_detection_model = './models/res10_300x300_ssd_iter_140000_fp16.caffemodel'
face_detection_proto = './models/deploy.prototxt.txt'
face_descriptor = './models/openface.nn4.small2.v1.t7'

# Loading deep neural network models
detector_model = cv2.dnn.readNetFromCaffe(face_detection_proto,face_detection_model)
descriptor_model = cv2.dnn.readNetFromTorch(face_descriptor)

In [ ]:
!dir gender_data

In [ ]:
# Showcasing sample image
img = cv2.imread('./gender_data/Female/131422.jpg.jpg')
cv2.imshow('sample',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def face_helper(image_path):
    img = cv2.imread(image_path)
    # Detect face image
    image = img.copy()
    h,w = image.shape[:2]
    img_blob = cv2.dnn.blobFromImage(image,1,(300,300), (104,177,123), swapRB = False, crop = False)

    # Set image to the model
    detector_model.setInput(img_blob)
    detections = detector_model.forward()

    if len(detections) > 0:
        i = np.argmax(detections[0,0,:,2]) # Considering face with max confidence score
        confidence = detections[0,0,i,2]

        if confidence > 0.5:
            box = detections[0,0,i,3:7] * np.array([w,h,w,h])
            (startx,starty,endx,endy) = box.astype('int')

            # Extracting face's features
            roi = image[starty:endy,startx:endx].copy()

            # Get the face's descriptors
            faceblob = cv2.dnn.blobFromImage(roi,1/255,(96,96), (0,0,0), swapRB = True, crop = True)
            descriptor_model.setInput(faceblob)
            vectors = descriptor_model.forward()            
            return vectors
    return None

# Getting Face Descriptors

In [ ]:
data = dict(data=[],label=[])

In [ ]:
folders = os.listdir('gender_data')
for folder in folders:
    filenames = os.listdir('gender_data/{}'.format(folder))
    for filename in filenames:
        try:
            vector = face_helper('./gender_data/{}/{}'.format(folder,filename))
            if vector is not None:
                data['data'].append(vector)
                data['label'].append(folder)
                print('Feature Extracted Successfully')
        except:
            pass 

In [ ]:
data.keys()

In [ ]:
pd.Series(data['label']).value_counts()

In [ ]:
# Save all the data
pickle.dump(data,open('face_gender_classifier.pickle', mode='wb'))